## Primer ejemplo con flujos de datos estructurados

#### Structured Streaming
Apache Spark incluye una API de procesamiento de flujos de datos de alto nivel, llamada [Structured Streaming](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html). 

- Structured Streaming es un motor de procesamiento de flujos de datos escalable y tolerante a fallos construida sobre el motor de Spark SQL.
- Una de las principales características es: podemos expresar las operaciones sobre los flujos de datos de la misma manera que expresaría un cálculo por lotes en datos estáticos.

En 'Structured streaming' un flujo de datos es tratado como una tabla que constantemente se está actualizando.

<img src="https://docs.databricks.com/_images/gsasg-spark-streaming-workflow.png" alt="image"/>

Cada elemento de datos que llega a la transmisión es como una nueva fila que se agrega a la tabla de entrada.

<img src="https://docs.databricks.com/_images/gsasg-spark-streaming-model.png" alt="image"/>

El objetivo de esta libreta es construir un ejemplo básico usando Structured Streaming. La meta es calcular métricas en tiempo real, como conteos continuos y conteos en ventanas en un flujo de acciones con marca de tiempo (por ejemplo, Abrir, Cerrar, etc.)

En esta libreta aprenderemos:
- Cargar una muestra de datos
- Inicializar un flujo de datos
- Iniciar un proceso de flujos de datos
- Hacer una consulta al flujo de datos

### Carga de datos

In [0]:
%fs ls /databricks-datasets/structured-streaming/events/

path name size dbfs:/databricks-datasets/structured-streaming/events/file-0.json file-0.json 72530 dbfs:/databricks-datasets/structured-streaming/events/file-1.json file-1.json 72961 dbfs:/databricks-datasets/structured-streaming/events/file-10.json file-10.json 73025 dbfs:/databricks-datasets/structured-streaming/events/file-11.json file-11.json 72999 dbfs:/databricks-datasets/structured-streaming/events/file-12.json file-12.json 72987 dbfs:/databricks-datasets/structured-streaming/events/file-13.json file-13.json 73006 dbfs:/databricks-datasets/structured-streaming/events/file-14.json file-14.json 73003 dbfs:/databricks-datasets/structured-streaming/events/file-15.json file-15.json 73007 dbfs:/databricks-datasets/structured-streaming/events/file-16.json file-16.json 72978 dbfs:/databricks-datasets/structured-streaming/events/file-17.json file-17.json 73008 dbfs:/databricks-datasets/structured-streaming/events/file-18.json file-18.json 73002 dbfs:/databricks-datasets/structured-streaming/events/file-19.json file-19.json 73014 dbfs:/databricks-datasets/structured-streaming/events/file-2.json file-2.json 73007 dbfs:/databricks-datasets/structured-streaming/events/file-20.json file-20.json 72987 dbfs:/databricks-datasets/structured-streaming/events/file-21.json file-21.json 72983 dbfs:/databricks-datasets/structured-streaming/events/file-22.json file-22.json 73009 dbfs:/databricks-datasets/structured-streaming/events/file-23.json file-23.json 72985 dbfs:/databricks-datasets/structured-streaming/events/file-24.json file-24.json 73020 dbfs:/databricks-datasets/structured-streaming/events/file-25.json file-25.json 72980 dbfs:/databricks-datasets/structured-streaming/events/file-26.json file-26.json 73002 dbfs:/databricks-datasets/structured-streaming/events/file-27.json file-27.json 73013 dbfs:/databricks-datasets/structured-streaming/events/file-28.json file-28.json 73005 dbfs:/databricks-datasets/structured-streaming/events/file-29.json file-29.json 72977 dbfs:/databricks-datasets/structured-streaming/events/file-3.json file-3.json 72996 dbfs:/databricks-datasets/structured-streaming/events/file-30.json file-30.json 73009 dbfs:/databricks-datasets/structured-streaming/events/file-31.json file-31.json 73008 dbfs:/databricks-datasets/structured-streaming/events/file-32.json file-32.json 72982 dbfs:/databricks-datasets/structured-streaming/events/file-33.json file-33.json 73033 dbfs:/databricks-datasets/structured-streaming/events/file-34.json file-34.json 72985 dbfs:/databricks-datasets/structured-streaming/events/file-35.json file-35.json 72974 dbfs:/databricks-datasets/structured-streaming/events/file-36.json file-36.json 73013 dbfs:/databricks-datasets/structured-streaming/events/file-37.json file-37.json 72989 dbfs:/databricks-datasets/structured-streaming/events/file-38.json file-38.json 72999 dbfs:/databricks-datasets/structured-streaming/events/file-39.json file-39.json 73013 dbfs:/databricks-datasets/structured-streaming/events/file-4.json file-4.json 72992 dbfs:/databricks-datasets/structured-streaming/events/file-40.json file-40.json 72986 dbfs:/databricks-datasets/structured-streaming/events/file-41.json file-41.json 73019 dbfs:/databricks-datasets/structured-streaming/events/file-42.json file-42.json 72986 dbfs:/databricks-datasets/structured-streaming/events/file-43.json file-43.json 72990 dbfs:/databricks-datasets/structured-streaming/events/file-44.json file-44.json 73018 dbfs:/databricks-datasets/structured-streaming/events/file-45.json file-45.json 72997 dbfs:/databricks-datasets/structured-streaming/events/file-46.json file-46.json 72991 dbfs:/databricks-datasets/structured-streaming/events/file-47.json file-47.json 73009 dbfs:/databricks-datasets/structured-streaming/events/file-48.json file-48.json 72993 dbfs:/databricks-datasets/structured-streaming/events/file-49.json file-49.json 73496 dbfs:/databricks-datasets/structured-streaming/events/file-5.json file-5.json 72998 dbfs:/databricks-datasets/structured-stre

Cada archivo json contiene un conjunto de líneas con dos atributos por línea: acción y tiempo.

In [0]:
%fs head /databricks-datasets/structured-streaming/events/file-0.json

[Truncated to first 65536 bytes]
{"time":1469501107,"action":"Open"}
{"time":1469501147,"action":"Open"}
{"time":1469501202,"action":"Open"}
{"time":1469501219,"action":"Open"}
{"time":1469501225,"action":"Open"}
{"time":1469501234,"action":"Open"}
{"time":1469501245,"action":"Open"}
{"time":1469501246,"action":"Open"}
{"time":1469501248,"action":"Open"}
{"time":1469501256,"action":"Open"}
{"time":1469501264,"action":"Open"}
{"time":1469501266,"action":"Open"}
{"time":1469501267,"action":"Open"}
{"time":1469501269,"action":"Open"}
{"time":1469501271,"action":"Open"}
{"time":1469501282,"action":"Open"}
{"time":1469501285,"action":"Open"}
{"time":1469501291,"action":"Open"}
{"time":1469501297,"action":"Open"}
{"time":1469501303,"action":"Open"}
{"time":1469501322,"action":"Open"}
{"time":1469501335,"action":"Open"}
{"time":1469501344,"action":"Open"}
{"time":1469501346,"action":"Open"}
{"time":1469501349,"action":"Open"}
{"time":1469501357,"action":"Open"}
{"time":1469501366,"action":"Open"}
{"time":1469501371,"action":"Open"}
{"time":1469501375,"action":"Open"}
{"time":1469501375,"action":"Open"}
{"time":1469501381,"action":"Open"}
{"time":1469501392,"action":"Open"}
{"time":1469501402,"action":"Open"}
{"time":1469501407,"action":"Open"}
{"time":1469501410,"action":"Open"}
{"time":1469501420,"action":"Open"}
{"time":1469501424,"action":"Open"}
{"time":1469501438,"action":"Open"}
{"time":1469501442,"action":"Close"}
{"time":1469501462,"action":"Open"}
{"time":1469501480,"action":"Open"}
{"time":1469501488,"action":"Open"}
{"time":1469501489,"action":"Open"}
{"time":1469501491,"action":"Open"}
{"time":1469501503,"action":"Open"}
{"time":1469501505,"action":"Open"}
{"time":1469501509,"action":"Open"}
{"time":1469501513,"action":"Open"}
{"time":1469501517,"action":"Open"}
{"time":1469501520,"action":"Open"}
{"time":1469501525,"action":"Open"}
{"time":1469501533,"action":"Open"}
{"time":1469501539,"action":"Open"}
{"time":1469501540,"action":"Open"}
{"time":1469501541,"action":"Open"}
{"time":1469501543,"action":"Open"}
{"time":1469501544,"action":"Open"}
{"time":1469501545,"action":"Close"}
{"time":1469501545,"action":"Open"}
{"time":1469501547,"action":"Open"}
{"time":1469501552,"action":"Open"}
{"time":1469501557,"action":"Open"}
{"time":1469501559,"action":"Open"}
{"time":1469501560,"action":"Open"}
{"time":1469501560,"action":"Open"}
{"time":1469501565,"action":"Open"}
{"time":1469501566,"action":"Open"}
{"time":1469501574,"action":"Open"}
{"time":1469501575,"action":"Open"}
{"time":1469501575,"action":"Open"}
{"time":1469501578,"action":"Open"}
{"time":1469501581,"action":"Open"}
{"time":1469501584,"action":"Open"}
{"time":1469501600,"action":"Open"}
{"time":1469501601,"action":"Open"}
{"time":1469501603,"action":"Open"}
{"time":1469501610,"action":"Open"}
{"time":1469501620,"action":"Open"}
{"time":1469501621,"action":"Open"}
{"time":1469501625,"action":"Open"}
{"time":1469501625,"action":"Close"}
{"time":1469501626,"action":"Open"}
{"time":1469501631,"action":"Open"}
{"time":1469501632,"action":"Open"}
{"time":1469501632,"action":"Open"}
{"time":1469501638,"action":"Open"}
{"time":1469501643,"action":"Open"}
{"time":1469501646,"action":"Open"}
{"time":1469501662,"action":"Open"}
{"time":1469501662,"action":"Open"}
{"time":1469501662,"action":"Open"}
{"time":1469501663,"action":"Open"}
{"time":1469501667,"action":"Open"}
{"time":1469501674,"action":"Open"}
{"time":1469501675,"action":"Open"}
{"time":1469501678,"action":"Close"}
{"time":1469501680,"action":"Open"}
{"time":1469501685,"action":"Open"}
{"time":1469501686,"action":"Open"}
{"time":1469501689,"action":"Open"}
{"time":1469501691,"action":"Open"}
{"time":1469501694,"action":"Open"}
{"time":1469501696,"action":"Close"}
{"time":1469501702,"action":"Open"}
{"time":1469501703,"action":"Open"}
{"time":1469501704,"action":"Open"}
{"time":1469501706,"action":"Open"}
{"time":1469501706,"action":"Open"}
{"time":1469501710,"action":"Open"}
{"time":1469501715,"action":"Open"}
{"

### Batch/Interactive Processing

Como podemos observar tenemos datos estáticos, los cuales van a utilizar para emular un flujo de datos. Para lograrlo vamos a leer un archivo a la vez (simulando un paso de tiempo).

In [0]:
from pyspark.sql.types import *

inputPath = "/databricks-datasets/structured-streaming/events/"

# Dado que ya conocemos que los datos están en json, se define un esquema para acelerar el procesamiento.
jsonSchema = StructType([ StructField("time", TimestampType(), True), StructField("action", StringType(), True) ])

# Creamos un dataframe estático para representar los archivos json
staticInputDF = (spark.read.schema(jsonSchema).json(inputPath))

#Visualizamos los datos
display(staticInputDF)

time action 2016-07-28T04:19:28.000+0000 Close 2016-07-28T04:19:28.000+0000 Close 2016-07-28T04:19:29.000+0000 Open 2016-07-28T04:19:31.000+0000 Close 2016-07-28T04:19:31.000+0000 Open 2016-07-28T04:19:31.000+0000 Open 2016-07-28T04:19:32.000+0000 Close 2016-07-28T04:19:33.000+0000 Close 2016-07-28T04:19:35.000+0000 Close 2016-07-28T04:19:36.000+0000 Open 2016-07-28T04:19:38.000+0000 Close 2016-07-28T04:19:40.000+0000 Open 2016-07-28T04:19:41.000+0000 Close 2016-07-28T04:19:42.000+0000 Open 2016-07-28T04:19:45.000+0000 Open 2016-07-28T04:19:47.000+0000 Open 2016-07-28T04:19:48.000+0000 Open 2016-07-28T04:19:49.000+0000 Open 2016-07-28T04:19:55.000+0000 Open 2016-07-28T04:20:00.000+0000 Close 2016-07-28T04:20:00.000+0000 Open 2016-07-28T04:20:01.000+0000 Open 2016-07-28T04:20:03.000+0000 Close 2016-07-28T04:20:07.000+0000 Open 2016-07-28T04:20:11.000+0000 Open 2016-07-28T04:20:12.000+0000 Close 2016-07-28T04:20:12.000+0000 Open 2016-07-28T04:20:13.000+0000 Close 2016-07-28T04:20:16.000+0000 Open 2016-07-28T04:20:23.000+0000 Close 2016-07-28T04:20:23.000+0000 Close 2016-07-28T04:20:23.000+0000 Open 2016-07-28T04:20:26.000+0000 Close 2016-07-28T04:20:30.000+0000 Close 2016-07-28T04:20:32.000+0000 Open 2016-07-28T04:20:32.000+0000 Open 2016-07-28T04:20:34.000+0000 Close 2016-07-28T04:20:36.000+0000 Open 2016-07-28T04:20:42.000+0000 Close 2016-07-28T04:20:42.000+0000 Open 2016-07-28T04:20:42.000+0000 Open 2016-07-28T04:20:48.000+0000 Close 2016-07-28T04:20:48.000+0000 Close 2016-07-28T04:20:48.000+0000 Open 2016-07-28T04:20:50.000+0000 Open 2016-07-28T04:20:52.000+0000 Open 2016-07-28T04:20:55.000+0000 Open 2016-07-28T04:20:55.000+0000 Open 2016-07-28T04:20:56.000+0000 Close 2016-07-28T04:20:56.000+0000 Close 2016-07-28T04:20:56.000+0000 Open 2016-07-28T04:20:59.000+0000 Open 2016-07-28T04:20:59.000+0000 Open 2016-07-28T04:21:02.000+0000 Close 2016-07-28T04:21:04.000+0000 Open 2016-07-28T04:21:08.000+0000 Open 2016-07-28T04:21:11.000+0000 Close 2016-07-28T04:21:13.000+0000 Open 2016-07-28T04:21:18.000+0000 Open 2016-07-28T04:21:19.000+0000 Open 2016-07-28T04:21:20.000+0000 Close 2016-07-28T04:21:22.000+0000 Open 2016-07-28T04:21:23.000+0000 Open 2016-07-28T04:21:27.000+0000 Close 2016-07-28T04:21:28.000+0000 Open 2016-07-28T04:21:31.000+0000 Close 2016-07-28T04:21:32.000+0000 Open 2016-07-28T04:21:32.000+0000 Open 2016-07-28T04:21:33.000+0000 Open 2016-07-28T04:21:34.000+0000 Close 2016-07-28T04:21:34.000+0000 Open 2016-07-28T04:21:35.000+0000 Close 2016-07-28T04:21:37.000+0000 Open 2016-07-28T04:21:38.000+0000 Close 2016-07-28T04:21:44.000+0000 Open 2016-07-28T04:21:46.000+0000 Close 2016-07-28T04:21:46.000+0000 Open 2016-07-28T04:21:48.000+0000 Close 2016-07-28T04:21:49.000+0000 Open 2016-07-28T04:21:50.000+0000 Close 2016-07-28T04:21:52.000+0000 Close 2016-07-28T04:21:52.000+0000 Open 2016-07-28T04:21:52.000+0000 Open 2016-07-28T04:21:53.000+0000 Open 2016-07-28T04:21:53.000+0000 Open 2016-07-28T04:21:56.000+0000 Close 2016-07-28T04:21:56.000+0000 Close 2016-07-28T04:21:57.000+0000 Close 2016-07-28T04:21:58.000+0000 Open 2016-07-28T04:21:59.000+0000 Close 2016-07-28T04:22:01.000+0000 Open 2016-07-28T04:22:06.000+0000 Close 2016-07-28T04:22:10.000+0000 Open 2016-07-28T04:22:11.000+0000 Close 2016-07-28T04:22:12.000+0000 Open 2016-07-28T04:22:14.000+0000 Close 2016-07-28T04:22:15.000+0000 Close 2016-07-28T04:22:15.000+0000 Open 2016-07-28T04:22:19.000+0000 Close 2016-07-28T04:22:20.000+0000 Close 2016-07-28T04:22:24.000+0000 Close 2016-07-28T04:22:24.000+0000 Close 2016-07-28T04:22:24.000+0000 Open 2016-07-28T04:22:30.000+0000 Open 2016-07-28T04:22:31.000+0000 Close 2016-07-28T04:22:31.000+0000 Open 2016-07-28T04:22:33.000+0000 Close 2016-07-28T04:22:39.000+0000 Open 2016-07-28T04:22:40.000+0000 Close 2016-07-28T04:22:43.000+0000 Open 2016-07-28T04:22:44.000+0000 Open 2016-07-28T04:22:49.000+0000 Close 2016-07-28T04:22:50.000+0000 Open 2016-07-28T04:22:55.000+0000 Open 2016-07-28T04:22:56.000+0000 Open 2016-07-28T04:22:58.000+0000

Iniciamos calculando el número de veces que aparece la acción de "open" y "close" en una ventana de una hora. Para hacerlo, agrupamos por la columna acción y sobre una hora para la columna de tiempo.

In [0]:
from pyspark.sql.functions import *      # for window() function

staticCountsDF = (staticInputDF.groupBy(staticInputDF.action, window(staticInputDF.time, "1 hour")).count())
staticCountsDF.cache()

# Guardamos el dataframe obtenido como una tabla
staticCountsDF.createOrReplaceTempView("static_counts")

Ahora, podemos directamente usar SQL para consultar la tabla. Podemos generar una línea de tiempo usando la tabla anterior

In [0]:
%sql select action, date_format(window.end, "MMM-dd HH:mm") as time, count from static_counts order by time, action

action time count Close Jul-26 03:00 11 Open Jul-26 03:00 179 Close Jul-26 04:00 344 Open Jul-26 04:00 1001 Close Jul-26 05:00 815 Open Jul-26 05:00 999 Close Jul-26 06:00 1003 Open Jul-26 06:00 1000 Close Jul-26 07:00 1011 Open Jul-26 07:00 993 Close Jul-26 08:00 989 Open Jul-26 08:00 1008 Close Jul-26 09:00 985 Open Jul-26 09:00 996 Close Jul-26 10:00 983 Open Jul-26 10:00 1000 Close Jul-26 11:00 1022 Open Jul-26 11:00 1007 Close Jul-26 12:00 1028 Open Jul-26 12:00 991 Close Jul-26 13:00 960 Open Jul-26 13:00 996 Close Jul-26 14:00 1028 Open Jul-26 14:00 1006 Close Jul-26 15:00 994 Open Jul-26 15:00 991 Close Jul-26 16:00 988 Open Jul-26 16:00 1020 Close Jul-26 17:00 984 Open Jul-26 17:00 992 Close Jul-26 18:00 1036 Open Jul-26 18:00 990 Close Jul-26 19:00 1001 Open Jul-26 19:00 1004 Close Jul-26 20:00 967 Open Jul-26 20:00 998 Close Jul-26 21:00 1035 Open Jul-26 21:00 1010 Close Jul-26 22:00 995 Open Jul-26 22:00 998 Close Jul-26 23:00 1036 Open Jul-26 23:00 997 Close Jul-27 00:00 950 Open Jul-27 00:00 1000 Close Jul-27 01:00 1008 Open Jul-27 01:00 998 Close Jul-27 02:00 1013 Open Jul-27 02:00 1004 Close Jul-27 03:00 971 Open Jul-27 03:00 992 Close Jul-27 04:00 1025 Open Jul-27 04:00 1014 Close Jul-27 05:00 989 Open Jul-27 05:00 995 Close Jul-27 06:00 987 Open Jul-27 06:00 986 Close Jul-27 07:00 1026 Open Jul-27 07:00 1016 Close Jul-27 08:00 982 Open Jul-27 08:00 998 Close Jul-27 09:00 1024 Open Jul-27 09:00 1002 Close Jul-27 10:00 990 Open Jul-27 10:00 992 Close Jul-27 11:00 1001 Open Jul-27 11:00 1006 Close Jul-27 12:00 1006 Open Jul-27 12:00 998 Close Jul-27 13:00 1035 Open Jul-27 13:00 994 Close Jul-27 14:00 986 Open Jul-27 14:00 1008 Close Jul-27 15:00 948 Open Jul-27 15:00 984 Close Jul-27 16:00 1018 Open Jul-27 16:00 1017 Close Jul-27 17:00 970 Open Jul-27 17:00 992 Close Jul-27 18:00 1020 Open Jul-27 18:00 1007 Close Jul-27 19:00 1036 Open Jul-27 19:00 995 Close Jul-27 20:00 969 Open Jul-27 20:00 1007 Close Jul-27 21:00 1025 Open Jul-27 21:00 1005 Close Jul-27 22:00 979 Open Jul-27 22:00 998 Close Jul-27 23:00 996 Open Jul-27 23:00 986 Close Jul-28 00:00 1011 Open Jul-28 00:00 1008 Close Jul-28 01:00 988 Open Jul-28 01:00 1000 Close Jul-28 02:00 1010 Open Jul-28 02:00 1001 Close Jul-28 03:00 1007 Open Jul-28 03:00 1000 Close Jul-28 04:00 993 Open Jul-28 04:00 996 Close Jul-28 05:00 960 Open Jul-28 05:00 825 Close Jul-28 06:00 671 Close Jul-28 07:00 191

### Stream Processing

Ahora que hemos analizado los datos de forma interactiva, vamos a convertir esto en una consulta de transmisión que se actualiza continuamente a medida que llegan los datos. 
Dado que solo tenemos un conjunto estático de archivos:
- Vamos a emular una secuencia de ellos leyendo un archivo a la vez, en el orden cronológico en que fueron creados. 
- La consulta que tenemos que escribir es más o menos la misma que la consulta interactiva anterior.

In [0]:
from pyspark.sql.functions import *

# Similar to definition of staticInputDF above, just using `readStream` instead of `read`
streamingInputDF = (spark.readStream                       
                    .schema(jsonSchema)               # Configuramos el esquema para los datos json
                    .option("maxFilesPerTrigger", 1)  # Tratamos una secuencia de archivos como un stream (tomamos un archivo a la vez)
                    .json(inputPath))

In [0]:
# Misma consulta como el dataframe estático
streamingCountsDF = (streamingInputDF.groupBy(streamingInputDF.action, window(streamingInputDF.time, "1 hour")).count())

#Ahora tenemos un dataframe de tipo flujo de datos
streamingCountsDF.isStreaming

Out[16]: True

Ahora podemos iniciar de la transmisión de los flujos de datos. En nuestro caso, queremos consultar de forma interactiva los recuentos (las mismas consultas que arriba), por lo que configuraremos el conjunto completo de recuentos de 1 hora para que estén en una tabla en memoria.

In [0]:
spark.conf.set("spark.sql.shuffle.partitions", "2")  # configuramos un número pequeño para ordenamiento

query = (streamingCountsDF.writeStream
        .format("memory")        # memory = almacenamos la tabla en memoria
        .queryName("counts")     # counts = nombre de la tabla en memoria
        .outputMode("complete")  # complete = todas las cuentas deben estar en la tabla
        .start())

La consulta (query) se mantiene con una consulta en streaming que está corriendo (in the background). Esta consulta está continuamente tomando un archivo y actualiza el conteo.En las gŕaficas dinámicas podemos observar el número de archivos que está siento procesados.

Una vez que algunos archivos han sido procesados, podemos hacer consultas de manera interactiva en la tabla que está en memoria.

In [0]:
from time import sleep
sleep(5)  # esperamos unos segundos a que el cómputo inicie

In [0]:
%sql select action, date_format(window.end, "MMM-dd HH:mm") as time, count from counts order by time, action

action time count Close Jul-26 03:00 11 Open Jul-26 03:00 179 Close Jul-26 04:00 344 Open Jul-26 04:00 1001 Close Jul-26 05:00 815 Open Jul-26 05:00 999 Close Jul-26 06:00 1003 Open Jul-26 06:00 1000 Close Jul-26 07:00 1011 Open Jul-26 07:00 993 Close Jul-26 08:00 989 Open Jul-26 08:00 1008 Close Jul-26 09:00 985 Open Jul-26 09:00 996 Close Jul-26 10:00 983 Open Jul-26 10:00 1000 Close Jul-26 11:00 1022 Open Jul-26 11:00 1007 Close Jul-26 12:00 1028 Open Jul-26 12:00 991 Close Jul-26 13:00 960 Open Jul-26 13:00 996 Close Jul-26 14:00 1028 Open Jul-26 14:00 1006 Close Jul-26 15:00 994 Open Jul-26 15:00 991 Close Jul-26 16:00 333 Open Jul-26 16:00 327

In [0]:
#query.stop()